In [ ]:
import numpy as np

from .primitive import Primitive
from .collider import Collider

from raypy.utils.constants import *


## Sphäre

Eine Sphäre (oft auch Kugel genannt) beschreibt eine runde Figur oder ihre Oberfläche. Wichtig dabei ist, dass jeder Punkt auf der Oberfläche die gleiche Distanz zum Mittelpunkt besitzt.


![Sphere](assets/Sphere.jpg)


Wird ein Lichtstrahl ausgesendet, muss berechnet werden ob und wie der Lichstrahl zur Späre steht. Hier interessiert vorallem der erste Auftreffpunkt. Es gibt dabei 3 Möglichkeiten wie der Lichtstrahl zur Späre steht.
- Er verfehlt sie.
- Er streift sie.
- Er durchquert sie.

![SphereIntersection](assets/IntersectionSphere.jpg)

Wird die Späre verfehlt, dann gibt es **keinen** Schnittpunkt.

Wird die Späre nur gestriffen, dann gibt es **genau einen** Schnittpunkt, unser erster Auftreffpunkt (rot umkreist)

Wird die Späre durchkreuzt, dann gibt es 2 Schnittpunkte, einen Eintrittspunkt und einen Austrittspunkt. In diesem Fall interessiert uns nur der erste Auftreffpunkt, also der Eintrittspunkt (rot umkreist).

Die Schnittpunkte haben exakt den Radius r als Distanz zum Mittelpunkt.

Bevor wir die Schnittpunkte berechnen schauen wir uns noch die unterschiedlichen Positionen des Ray Ursprungs an. Der Ursprung des Rays kann 3 Positionen haben:

![OriginRay](assets/OriginRay.jpg)

Möchte man nun die Schnittpunkte berechnen, benötigt man die Formel für den Ray:

$$P(t)=\vec{O}+\vec{R}*t$$

Aus dieser Formel können wir nun die Formeln für die Schnittpunkte aufstellen.

$$P_1(t)=\vec{O}+\vec{R}*t_1$$

$$P_2(t)=\vec{O}+\vec{R}*t_2$$

Wir suchen nun die Werte $t_1$ und $t_2$. In der Abbildung oben sieht man, dass $t_1$ den Abstand des Ursprungs des Rays $O$ zum Punkt $P_1$ beschreibt und $t_2$ ist der Abstand des Ursprungs $O$ zum Punkt $P_2$.

Dafür brauchen wir die Gleichung der Sphäre.

Liegt der Radius im Ursprung $U (0,0,0)$ und der Punkt auf der Oberfläche so ergibt sich die Gleichung:
$$x^2 + y^2 + z^2 = r^2$$

Liegt der Radius nicht im Ursprung sondern im Punkt $M (M_x, M_y, M_z)$ ergibt sich:
$$(x-M_x)^2 + (y-M_y)^2 + (y-M_z)^2 = r^2$$

$x, y, z$ gehören in diesem Beispiel zu einem Punkt P, wodurch wir die Formel vereinfachen können:
$$(P-M) * (P-M) = (x-M_x)^2 + (y-M_y)^2 + (y-M_z)^2 = r^2$$

Wir suchen jetzt aber nicht irgendein Punkt P, sondern einen Punkt P der auf unserem Lichtstrahl $P(t)=\vec{O}+\vec{R}*t$ liegt. Daher setzen wir nun für P unsere Lichtstrahl-Gleichung ein:
$$(\vec{O}+\vec{R}*t-M) * (\vec{O}+\vec{R}*t-M) = r^2$$

Formen wir die quadratische Gleichung um erhalten wir vereinfacht:
$$t^2 * \vec{R} + t * 2*\vec{R}*(\vec{O}-M) + (\vec{O}-M) * (\vec{O}-M) - r^2 = 0$$

Wie man sieht können wir jetzt einfach die Mitternachtsformel mit

$ a = \vec{R}$,

$ b = 2*\vec{R}*(\vec{O}-M)$ und

$c = (\vec{O}-M) * (\vec{O}-M) - r^2$

anwenden, um unsere Werte $t_1$ und $t_2$ auszurechnen.


Anhand des Ergebnis der Mitternachtsformel kann auch bestimmt werden, wie viele Schnittpunkte es gibt:
- positiver Wert unter der Wurzel: **zwei** Schnittpunkte
- 0 unter der Wurzel: **ein** Schnittpunkt
- negativer Wert unter der Wurzel: **kein** Schnittpunkt

Nachfolgend sind 2 Klassen, um eine Späre/Kugel darzustellen.

Die erste Klasse ist die **primitive** Darstellung, bedeutet damit werden die Parameter der Kugel festgelegt, wie Mittelpunkt, Radius, Material, ...

Die zweite Klasse implementiert die Aktionen die beim Aussenden eines Rays abgehandelt werden müssen, bedeutet das Ausrechen des Schnittpunkts.

In [ ]:
class Sphere(Primitive):
    def __init__(self, center, material, radius, max_ray_depth=5, shadow=True):
        super().__init__(center, material, max_ray_depth, shadow)
        self.collider_list += [SphereCollider(assigned_primitive=self, center=center, radius=radius)]
        self.bounded_sphere_radius = radius

    def get_uv(self, hit):
        return hit.collider.get_uv(hit)


In [ ]:
class SphereCollider(Collider):
    def __init__(self,  radius, **kwargs):
        super().__init__(**kwargs)
        self.radius = radius

    def intersect(self, origin, direction):
        b = 2 * direction.dot(origin - self.center)
        c = self.center.square_length() + origin.square_length() - 2 * self.center.dot(origin) - (self.radius * self.radius)
        disc = (b ** 2) - (4 * c)
        sq = np.sqrt(np.maximum(0, disc))
        h0 = (-b - sq) / 2
        h1 = (-b + sq) / 2
        h = np.where((h0 > 0) & (h0 < h1), h0, h1)
        pred = (disc > 0) & (h > 0)
        M = (origin + direction * h)
        NdotD = ((M - self.center) * (1. / self.radius)).dot(direction)

        pred1 = (disc > 0) & (h > 0) & (NdotD > 0)
        pred2 = (disc > 0) & (h > 0) & (NdotD < 0)
        pred3 = True

        # return an array with hit distance and the hit orientation
        return np.select([pred1, pred2, pred3],
                         [[h, np.tile(UPDOWN, h.shape)], [h, np.tile(UPWARDS, h.shape)], FARAWAY])

    def get_normal(self, hit):
        return (hit.point - self.center) / self.radius